In [46]:
import configparser
import os.path as op

CP = configparser.ConfigParser()
CP.read_file(open(op.join(op.expanduser('~'), '.aws', 'credentials')))
CP.sections()
aws_access_key = CP.get('hcp', 'AWS_ACCESS_KEY_ID')
aws_secret_key = CP.get('hcp', 'AWS_SECRET_ACCESS_KEY')

def attach_keys(arr):
    return [(e, aws_access_key, aws_secret_key) for e in arr]

In [47]:
def afq_hcp_retest(args):
    import fury
    import xvfbwrapper
    import logging
    import os.path as op
    import s3fs

    from AFQ.data import fetch_hcp
    import AFQ.api as api
    
    logging.basicConfig(level=logging.INFO)
    log = logging.getLogger(__name__)  
    
    fs = s3fs.S3FileSystem()
    
    subject = args[0]
    aws_access_key = args[1]
    aws_secret_key = args[2]

    _, hcp_1200_bids = fetch_hcp(
        [subject],
        profile_name=False,
        study='HCP_1200',
        aws_access_key_id=aws_access_key,
        aws_secret_access_key=aws_secret_key)

    _, hcp_retest_bids = fetch_hcp(
        [subject],
        profile_name=False,
        study='HCP_Retest',
        aws_access_key_id=aws_access_key,
        aws_secret_access_key=aws_secret_key)


    
    myafq_1200 = api.AFQ(
        hcp_1200_bids,
        seg_suffix='aseg',
        min_bval=990,
        max_bval=1010,
        b0_threshold=50,
        reg_template="mni_T1",
        reg_subject="power_map",
        wm_criterion=0.2,
        #tracking_params = {"odf_model": "DKI"},
        virtual_frame_buffer=True)
        #scalars=["dti_fa", "dti_md", "dki_fa", "dki_md"])

    myafq_retest = api.AFQ(
        hcp_retest_bids,
        seg_suffix='aseg',
        min_bval=990,
        max_bval=1010,
        b0_threshold=50,
        reg_template="mni_T1",
        reg_subject="power_map",
        wm_criterion=0.2,
        #tracking_params = {"odf_model": "DKI"},
        virtual_frame_buffer=True)
        #scalars=["dti_fa", "dti_md", "dki_fa", "dki_md"])
    
    myafq_1200.export_registered_b0()
    myafq_1200.get_template_xform()
    myafq_1200.export_bundles()
    myafq_1200.get_tract_profiles()
    myafq_1200.viz_bundles(export_as_gif=True)
    myafq_1200.viz_ROIs(export_as_gif=True)
    fs.put(op.join(hcp_1200_bids, "afq"), "profile-hcp/hcp_reliability/hcp_1200", recursive=True)

    myafq_retest.export_registered_b0()
    myafq_retest.get_template_xform()
    myafq_retest.export_bundles()
    myafq_retest.get_tract_profiles()
    myafq_retest.viz_bundles(export_as_gif=True)
    myafq_retest.viz_ROIs(export_as_gif=True)
    fs.put(op.join(hcp_retest_bids, "afq"), "profile-hcp/hcp_reliability/hcp_retest", recursive=True)


In [48]:
subjects_num = [
    169343,
    177746,
    195041,
    185442,
    433839,
    144226,
    200614,
    204521,
    601127,
    877168,
]
subjects = [str(i) for i in subjects_num] 
args = attach_keys(subjects)
#afq_hcp_api(args[0])

In [49]:
#afq_hcp_retest(attach_keys(['169343'])[0])

In [50]:
import cloudknot as ck
ck.set_region('us-east-1')

In [52]:
knot = ck.Knot(name='hcp_api-64gb-20731-0',
              func=afq_hcp_retest,
              base_image='libglxvfb:1',
              image_github_installs="https://github.com/36000/pyAFQ.git@reg_fixes",
              pars_policies=('AmazonS3FullAccess',),
              volume_size=64,
              memory=64000)

In [53]:
result_futures = knot.map(args)

In [63]:
knot.view_jobs()

Job ID              Name                        Status   
---------------------------------------------------------
e18e0bfe-5ae2-4910-b1d1-c1b9f7dd9641        hcp-api-64gb-20731-0-0        FAILED   


In [64]:
j0 = knot.jobs[0]
j0.status

{'status': 'FAILED',
 'statusReason': 'Array Child Job failed',
 'attempts': [],
 'arrayProperties': {'statusSummary': {'STARTING': 0,
   'FAILED': 7,
   'RUNNING': 0,
   'SUCCEEDED': 3,
   'RUNNABLE': 0,
   'SUBMITTED': 0,
   'PENDING': 0},
  'size': 10}}

In [51]:
knot.clobber(clobber_pars=True, clobber_repo=True, clobber_image=True)

In [ ]:
subjects = [169343, 204521, 433839]